In [1]:
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("1706.03762.pdf")
data = loader.load()
file_content = '\n'.join([d.page_content for d in data])

In [2]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=0, separator='\n')
texts = text_splitter.create_documents([file_content])

In [3]:
texts

[Document(page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable a

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)
template = """You are a helpful assistant.
You are given a scientific paper and asked questions about it.
"""
system_template_prompt = SystemMessagePromptTemplate.from_template(template)
map_human_template = """TEXT FROM THE PAPER:

{text}

QUESTION: Extract the sections from the text and return each section's header and a concise summary.

ANSWER:"""

map_human_template_prompt = HumanMessagePromptTemplate.from_template(map_human_template)
map_prompt = ChatPromptTemplate.from_messages([system_template_prompt, map_human_template_prompt])

combine_human_template = """TEXT FROM THE PAPER:

{text}

QUESTION: Extract the sections from the text and return each section's header and a concise summary.

ANSWER:"""
combine_human_template_prompt = HumanMessagePromptTemplate.from_template(combine_human_template)
combine_prompt = ChatPromptTemplate.from_messages([system_template_prompt, combine_human_template_prompt])

map_prompt.format_prompt(text=texts[0].page_content).to_messages()


[SystemMessage(content='You are a helpful assistant.\nYou are given a scientific paper and asked questions about it.\n', additional_kwargs={}),
 HumanMessage(content="TEXT FROM THE PAPER:\n\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

In [8]:
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True, map_prompt=map_prompt, combine_prompt=combine_prompt)              

In [11]:
resp = chain.run(texts[0:4])



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
System: You are a helpful assistant.
You are given a scientific paper and asked questions about it.

Human: TEXT FROM THE PAPER:

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence

In [81]:
print(resp)

Section 1: Introduction - Introduces the Transformer, a network architecture based solely on attention mechanisms, which outperforms existing models in quality, parallelizability, and training time.

Section 2: Background - Discusses the limitations of existing sequence transduction models based on recurrent or convolutional neural networks and introduces the attention mechanism and its use in connecting the encoder and decoder in sequence transduction models.

Section 3: Model Architecture - Describes the Transformer architecture, which consists of an encoder and decoder, both based solely on attention mechanisms, and explains the use of multi-head attention and position-wise feed-forward networks in the Transformer.

Section 4: Training - Discusses the training process for the Transformer, including the use of label smoothing and a custom learning rate schedule.

Section 5: Results - Presents the results of experiments on two machine translation tasks, showing that the Transformer ou